In [1]:
from langchain_community.document_loaders import PyMuPDFLoader, UnstructuredPDFLoader
from REMOVED_SECRET import Document as LangchainDocument
from langchain_community.vectorstores import FAISS
from REMOVED_SECRET import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import torch
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, TextStreamer
from ragatouille import RAGPretrainedModel
import nltk
from transformers import AutoTokenizer, AutoModelForCausalLM
from rag_for_notebook_sunday import CustomTextGenerationPipeline
import REMOVED_SECRET as F

In [17]:
custom_pipeline = CustomTextGenerationPipeline(model_id="microsoft/Phi-3-mini-4k-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


In [18]:
prompt = "Hello, how are you?"

generated_sequence, all_logits = custom_pipeline(prompt)


#check if logits require gradients
print(f"Logits requires_grad: {all_logits.requires_grad}")



/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



- Hey there, I feel like a real adventurer! I
Logits requires_grad: True


In [19]:
import torch
import REMOVED_SECRET as F
print(f"all_logits shape: {all_logits.shape}")
# Assuming all_logits is your logits tensor
#batch_size, sequence_length, vocab_size = all_logits.size()
sequence_length, vocab_size = all_logits.size()
# Example target tensor: must be of shape (batch_size * sequence_length)
# Replace this with your actual target IDs. Here, we're simulating with random targets
target = torch.randint(0, vocab_size, (sequence_length,)).to(all_logits.device)
# Flatten the logits for loss computation
#logits_for_loss = all_logits.view(-1, vocab_size)  # shape: (batch_size * sequence_length, vocab_size)
#print(f"logits_for_loss shape: {logits_for_loss.shape}")
print(f"target shape: {target.shape}")
# Calculate the cross-entropy loss
loss = F.cross_entropy(all_logits, target)

# Backpropagation to compute gradients
loss.backward()

print(f"Gradients computed for logits: {all_logits.grad is not None}")


all_logits shape: torch.Size([1, 448896])
target shape: torch.Size([1])
Gradients computed for logits: False


/tmp/ipykernel_18384/2690611828.py:20: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(f"Gradients computed for logits: {all_logits.grad is not None}")
